# Notebook contains:

* Timing tests notes
* Plotting tool to compare different paths, and plot time logs
* Fitting piecewise linear function
* reading in the tif with weights
* Faster edge convolutions
* read start and dest from tifs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# import networkx as nx
import os
from PIL import Image
import rasterio
import time
import json
from graph_tool.all import *

In [ ]:
from power_planner.utils.utils import *

## Next meeting:

* path straightening in constraints notebook with algorithm is not an actual straightening

### Explain pipeline:

* Get hard constraints --> binary corridor, + water etc
* Add one node to the graph for each raster cell in the corridor
* Define donut around each cell
* Add edges between each point and the point in the donut around it, with weights:
    * First option: sum of cost of both cells of these two nodes
    * Second option: sum of cost of all cells inbetween --> Bresenham line (other notebook)
* Add start and end node, and connect to the top x and bottom x points of the corridor with weight 0
* Run shortest path algorithm (bellman ford)

So far: downscaling with different factors

### Runtimes:

**Scale by 5 --> 20 poins in donut**
* edge convolutions: {'init_graph': 0.03, 'add_nodes': 1.676, **'edge_list (per convolution!)': 0.9, 'add_edges (per shift)': 0.0045**, 'start_end_vertex': 0.004, 'shortest_path': 0.363}
* node adding: {'init_graph': 0.03, 'add_nodes': 1.376, **'edge_list': 0.00245, 'add_edges': 0.00665,** 'start_end_vertex': 0.005, 'shortest_path': 0.236}
* old edge version {'init_graph': 0.031, 'add_nodes': 1.481, **'edge_list': 0.198, 'add_edges': 0.045**, 'start_end_vertex': 0.003, 'shortest_path': 0.216}

--> add adges even longer if list not split?!, improved node constraints by factor of 80

**networkx scale 5:**
* bellman ford: {'init_graph': 0.037, 'add_nodes': 0.03, 'edge_list': 0.002, **'add_edges': 0.2996**, 'start_end_vertex': 0.007, **'shortest_path': 2.961**}
* dijkstra: {'init_graph': 0.036, 'add_nodes': 0.03, 'edge_list': 0.0020000000000000005, 'add_edges': 0.2926000000000001, 'start_end_vertex': 0.004, 'shortest_path': 0.633}

--> graph-tool speed up: shortest path 10 times (bellman ford), 2 times (dijkstra), adding edges 60 times

**Scale by 2 --> 156 point in donut**
* edge convolutions with **faster convolutions**: {'init_graph': 0.233, 'add_nodes': 0.005, 'concatenate': 0.328, 'add_edges': 0.025, **'edge_list': 0.15**, 'start_end_vertex': 0.015, 'shortest_path': 27.296}
* edge convolutions: {'init_graph': 0.198, 'add_nodes': 9.569, **'edge_list': 6.908, 'add_edges': 0.0273**, 'start_end_vertex': 0.034, 'shortest_path': 32.844}
* node adding: {'init_graph': 0.202, 'add_nodes': 9.458, **'edge_list': 0.0092, 'add_edges': 0.024,** 'start_end_vertex': 0.017, 'shortest_path': 7.76}
* node adding graph tool dijkstra: {'init_graph': 0.274, 'add_nodes': 0.006, 'edge_list': 0.010, 'add_edges': 0.029, 'start_end_vertex': 0.018, **'shortest_path': 0.822**}
* old edge version: {'init_graph': 0.206, 'add_nodes': 10.979, **'edge_list': 1.154, 'add_edges': 0.314,** 'start_end_vertex': 0.032, 'shortest_path': 7.492}
* networkx dijkstra: {'init_graph': 0.212, 'add_nodes': 0.156, 'edge_list': 0.01, **'add_edges': 3.08,** 'start_end_vertex': 0.065, **'shortest_path': 41.075**}

--> speed up of node constraint by factor of 125
--> graph tool speedup: 6 times for shortest path, 

**Whole instance: 428643 nodes --> around 263 615 445 edges**
* dijkstra does not work because out of memory, time is {'init_graph': 0.624, 'add_nodes': 0.021, 'edge_list': 0.065,**'add_edges': 1.179**, 'start_end_vertex': 0.61,**'shortest_path': 611.761**}
* with adding all edges at ones (bellman ford graph-tool): crashed 

**Directed graph:**
* speed up for shortest path by a lot! 
* scale 2 directed: 0.5 
* {'init_graph': 0.218, 'add_nodes': 0.005, 'add_edges': 0.024, 'edge_list': 0.12, **'shortest_path': 0.518**}
* scale 2 undirected: 8

**Other observations:**
* graph-tool becomes slower for adding edges, the more edges are in the graph already
* graph-tool with adding all edges at once: {'init_graph': 0.211, 'add_nodes': 0.005, 'add_edges': 0.026, 'edge_list': 0.017, 'start_end_vertex': 0.007, 'shortest_path': 6.554}
* **add_nodes is much faster when no generator is returned (and seems to work just as fine**


### NEW TEST:
* whole instance {'init_graph': 0.626, 'add_nodes': 0.003, 'add_edges': 0.008, 'edge_list': 0.025, 'shortest_path': 6.268}

### Space efficiency:

* with dijkstra my laptop crashed, with bellman ford it's okay (for full instance --> 249 000 000 edges)
* scaled 2 still fine 15 912 513 edges

### Discuss:

* Angle constraints: 
    * Path straightening (other notebook) is actually sampling fewer points
    * Line graph takes m*1/3 d

### Questions:

Pipeline so far:

* Normalization: 0-1 normalization for cost surface?
* Bresenham line makes sense?
* Start and end node given? first row?
* networkx version

Constraints:

* Hard constraints: Distance from towns etc? Water?
* View on towers / cables constraint?
* Height of towers
* Monetary costs? --> node costs in addition, so it's not placing too many towers

How to use other tifs?? - Excel sheet

### meeting:

* infinity costs on edges for NaNs
* function to compute resistance: multiply raster tifs with weights
* the more negative the value, the more suitable
* completely forbidden: NaN or infinite
* MCDA negative values - but zickszack

Todo:
* flip coordinates

## Runtimes Linegraph

* Line graph: {'init_graph': 0.0, 'add_nodes': 1.7634360790252686, 'add_edges': 0.023, 'edge_list': 0.079, 'shortest_path': 15.495} for 878 iterations --> add_edges ca. 90 seconds
* normal graph: {'init_graph': 0.038, 'add_nodes': 0.001, 'add_edges': 0.003, 'edge_list': 0.002, 'shortest_path': 0.162}
* line graph from file:{'init_graph': 9.354, 'add_nodes': 0.028, 'add_edges': 48.366, 'edge_list': 902.766, 'shortest_path': 4.559}

## Bellman Ford:

* Normal bellman ford: for all nodes for all edges --> in each step, check all edges
* for normal BF: We know that after iteration N, if there is a path from source to target with N edges, then this is a shortest path with N edges
* how does this help? Idea: Approximate max number of pylons X. Then after X iterations, if there is a path of length X, it is the shortest path --> we could check all paths of length <= X, and take the shortest one of those.

Problem: ITERATING THROUGH ALL EDGES is very expensive

* very slow already for 450 000
* other implementations just work with checking neighbors of current vertex --> SPFA implemented in networkx and probably also graph-tool
* https://pdfs.semanticscholar.org/d75a/87a3232bef187e710ef59e202816187c8669.pdf try this: speed up of SPFA

Idea: do SPFA, stop if target was considered --> gives same result as before

# Plotting tool

In [ ]:
compare_path = "../outputs/compare" # path_whole_instance_infos.json"

In [ ]:
paths = []
files = []
for f in os.listdir(compare_path):
    if f[-4:]=="json":
        print(f)
        with open(os.path.join(compare_path,f),"r") as infile:
            infos = json.load(infile)
        paths.append(infos["path_cells"])
        files.append(f)

In [ ]:
plt.figure(figsize=(20,10))
for p, f in zip(paths, files):
    p_arr = np.array(p)
    plt.scatter(p_arr[:,1], p_arr[:,0], label = f)
    print(f, "path length:", len(p))
plt.legend()
plt.show()

### Observations:

* Way more nodes in edge cost path
* Combining both --> edge cost seem to matter more (still 61 vs 52 nodes) --> would need to weight them somehow
* idea: adapt donut

### check different edge costs

In [ ]:
import pickle

In [ ]:
info_path = "../outputs/path_15860_infos.json"
instance_path = "tifs_new/data_dump_5.dat"

In [ ]:
with open(info_path,"r") as infile:
    infos = json.load(infile)

In [ ]:
edgecosts = infos["edgecosts"]
edgecosts.insert(0, [0,0])
edgecosts.append([0,0])

In [ ]:
with open(instance_path, "rb") as infile:
    (instance, instance_corr, start_inds, dest_inds) = pickle.load(infile)

In [ ]:
scaled_path = np.asarray(infos["path_cells"]) // 5

In [ ]:
for i in range(len(scaled_path)-1):
    (h,j) = tuple(scaled_path[i])
    (k,l) = tuple(scaled_path[i+1])
    print(scaled_path[i], scaled_path[i+1], edgecosts[i])
    print(instance[h,j], instance[k,l])

In [ ]:
def plot_path_costs(instance, path, edgecosts, out_path = None, buffer=1):
    expanded = np.expand_dims(instance, axis=2)
    expanded = np.tile(expanded, (1, 1, 3))  # overwrite instance by tiled one
    
    edgecosts = np.asarray(edgecosts)
    env_costs = edgecosts[:,1] # np.sum(edgecosts, axis=1) # 
    normed_env_costs = (env_costs-np.min(env_costs))/(np.max(env_costs)-np.min(env_costs))
    # colour nodes in path in red
    for i, (x, y) in enumerate(path):
        # print(edgecosts[i])
        val = edgecosts[i,0]
        expanded[x - buffer:x + buffer + 1,
                 y - buffer:y + buffer + 1] = [0.9, 1-normed_env_costs[i], 0.2]  # colour red

    plt.figure(figsize=(25, 15))
    plt.imshow(np.swapaxes(expanded, 1,0), origin="upper")
    if out_path is not None:
        plt.savefig(out_path)
    else:
        plt.show()

In [ ]:
# Test: plot on instance
plot_path_costs(instance, scaled_path, edgecosts, out_path = "angle_costs.png")

### Timings

In [ ]:
json_file_path = "../outputs/path_37054_directed_startend_infos.json"
with open(os.path.join(json_file_path),"r") as infile:
    infos = json.load(infile)

In [ ]:
plt.plot(infos["time_logs"]["edge_list_times"])
plt.show()

## Is adding edges faster when nodes are first added

In [ ]:
g = Graph()

In [ ]:
_ = g.add_vertex(200001)

In [ ]:
import time

In [ ]:
edges = [[i,i+1] for i in range(200000)]

In [ ]:
tic = time.time()
g.add_edge_list(edges)
print(time.time()-tic)

## Piecewise linear fit

In [ ]:
import pwlf

In [ ]:
x = p_arr[:,0]
y = p_arr[:,1]

In [ ]:
my_pwlf = pwlf.PiecewiseLinFit(x,y)
breaks = my_pwlf.fit(7)

In [ ]:
x_hat = np.linspace(x.min(), x.max(), 100)
y_hat = my_pwlf.predict(x_hat)

plt.figure(figsize=(20,10))
plt.plot(x, y, 'o')
plt.plot(x_hat, y_hat, '-')
plt.show()

## Weighted constraint

In [ ]:
path_files = "tifs_new"

In [ ]:
weights = pd.read_csv(os.path.join(path_files,"layer_weights.csv"))

In [ ]:
def read_tif(path):
    with rasterio.open(path, 'r') as ds:
        print(path)
        arr = ds.read()
    return arr[0]

### Forbidden ones

In [ ]:
def get_hard_constraints(path_files, weights):
    hard_cons_rows = weights[weights["weight"]=="Forbidden"]
    hard_constraints = np.asarray([read_tif(os.path.join(path_files, "tif_layers", fname+".tif")) for fname in hard_cons_rows["Layer Name"]])
    # set to zero
    for hard in hard_constraints:
        plt.imshow(hard)
        plt.show()
    hard_constraints -= np.min(hard_constraints)
    hard_constraints = np.all(hard_constraints.astype(int), axis=0)
    return hard_constraints

In [ ]:
hard_constraints = get_hard_constraints(path_files, weights)

### Other ones

In [ ]:
weights = weights.dropna()

In [ ]:
layers = weights[weights["weight"]!="Forbidden"] # weights_wo_nan

In [ ]:
GENERAL_SHAPE = (1313, 1511)

In [ ]:
cost_sum_arr = np.zeros(GENERAL_SHAPE)
for fname, weight in zip(layers["Layer Name"], layers["weight"]):
    file_path = os.path.join(path_files, "tif_layers", fname+".tif")
    if os.path.exists(file_path):
        costs = read_tif(file_path)
    costs = np.absolute(normalize(costs)-1)
    print(np.min(costs), np.max(costs))
    cost_sum_arr = cost_sum_arr + costs * int(weight)

In [ ]:
plt.imshow(normalize(cost_sum_arr))
plt.colorbar()
plt.show()

## Start and end

In [ ]:
path_files = "tif_ras_buf"

In [ ]:
with rasterio.open(os.path.join(path_files, "corridor/CORRIDOR_BE.tif")) as dataset:
    print(dataset.width)
    print(dataset.bounds)
    transform_matrix = dataset.transform
    arr = dataset.read()
    print(arr.shape)
    print(dataset.crs)

In [ ]:
a = np.zeros((20,20))

In [ ]:
a[:5, 1] = 1
a[4, 1:8]=1
a[4, 10]=1
a[7:15, 10] = 1
a[14, 10] = 1
a[14, 13:15] = 1
a[14:, 14] = 1

start = [0,1]
end = [19, 14]
a[start[0], start[1]] = 2

In [ ]:
plt.imshow(a)
plt.show()

In [ ]:
arr2 = read_tif(os.path.join(path_files, "corridor/CORRIDOR_BE.tif"))

In [ ]:
corridor = arr2!=9999

In [ ]:
together = corr.astype(int)*hard_constraints

In [ ]:
together.shape

In [ ]:
np.pad(together, ((1,1), (1,1))).shape

In [ ]:
corr = np.zeros(together.shape)
corr[40:1260, 200:1000] =1

In [ ]:
x_inds, y_inds = np.where(corr)
x_len, y_len = corr.shape
min_dist = min([x_inds[0], x_len-x_inds[-1], y_inds[0], y_len-y_inds[-1]])

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(hard_constraints)# [600:800, 300:720]) # np.swapaxes(arr[0], 0,1)) # , origin="lower")
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
together[650:]
plt.imshow(together[650:680, 690:720]) # np.swapaxes(arr[0], 0,1)) # , origin="lower")
plt.show()

In [ ]:
import shapefile
sf = shapefile.Reader("tifs_new/Destination")

In [ ]:
shapes = sf.shapes()

In [ ]:
shapes[0].points

In [ ]:
with open("../../outputs/path_37054_directed_startend_infos.json", "r") as infile:
    coords = json.load(infile)

In [ ]:
from pyproj import Proj, transform
inProj = Proj(init=dataset.crs)
outProj = Proj(init='epsg:4326')
x1,y1 =126131.1214, 178356.2068
x2,y2 = transform(inProj,outProj,x1,y1)
transformed = []
for point in coords["path_cells"]:
    trans = transform_matrix * point
    x2,y2 = transform(inProj,outProj,trans[0],trans[1])
    transformed.append([y2,x2])

In [ ]:
df.to_csv("test.csv")

In [ ]:
df = pd.DataFrame(transformed)

In [ ]:
df = df.rename(columns={"0": "Lat", "1": "Long"})

### Convert to correct coordinates

In [ ]:
coords_arr = np.asarray(coords["path_cells"])
switched = np.roll(coords_arr, 1, axis=1)
coords_arr = np.asarray([list(transform_matrix*switched[i]) for i in range(len(switched))])

In [ ]:
df = pd.DataFrame(coords_arr, columns=["X", "Y"])
# df.to_csv("converted_coords.csv")

## Optimize convolution

In [ ]:
import numpy as np

from power_planner.utils.utils import get_half_donut


def bresenham_line(x0, y0, x1, y1):
    """
    find pixels on line between two pixels
    https://stackoverflow.com/questions/50995499/generating-pixel-values-of-line-connecting-2-points
    """
    steep = abs(y1 - y0) > abs(x1 - x0)
    if steep:
        x0, y0 = y0, x0
        x1, y1 = y1, x1

    switched = False
    if x0 > x1:
        switched = True
        x0, x1 = x1, x0
        y0, y1 = y1, y0

    if y0 < y1:
        ystep = 1
    else:
        ystep = -1

    deltax = x1 - x0
    deltay = abs(y1 - y0)
    error = -deltax / 2
    y = y0

    line = []
    for x in range(x0, x1 + 1):
        if steep:
            line.append([y, x])
        else:
            line.append([x, y])

        error = error + deltay
        if error > 0:
            y = y + ystep
            error = error - deltax
    if switched:
        line.reverse()
    return line


def get_kernel(shifts):
    """
    Get all kernels describing the path of the edges in a discrete raster
    :param shifts: possible circle points
    :returns kernel: all possible kernels (number of circle points x upper x upper)
    :returns posneg: a list indicating whether it is a path to the left (=1) or to the right(=0)
    """
    upper = np.amax(np.absolute(shifts)) + 1
    posneg = []
    kernel = np.zeros((len(shifts), upper, upper))

    for i, shift in enumerate(shifts):
        if shift[1] < 0:
            posneg.append(1)
            line = bresenham_line(0, upper - 1, shift[0], upper - 1 + shift[1])
        else:
            posneg.append(0)
            line = bresenham_line(0, 0, shift[0], shift[1])
        # add points of line to the kernel
        for (j, k) in line:
            kernel[i, j, k] += 1
    return kernel, posneg


def convolve(img, kernel, neg=0):
    """
    Convolve a 2d img with a kernel, storing the output in the cell 
    corresponding the the left or right upper corner 
    :param img: 2d numpy array
    :param kernel: kernel (must have equal size and width)
    :param neg: if neg=0, store in upper left corner, if neg=1, store in upper 
    right corner
    :return convolved image of same size
    """
    k_size = len(kernel)
    if neg:
        padded = np.pad(img, ((0, k_size - 1), (k_size - 1, 0)))
    else:
        padded = np.pad(img, ((0, k_size), (0, k_size)))
    # print(padded.shape)
    convolved = np.zeros(img.shape)
    w, h = img.shape
    for i in range(0, w):
        for j in range(0, h):
            patch = padded[i:i + k_size, j:j + k_size]
            convolved[i, j] = np.sum(patch * kernel)
    return convolved


def angle(path):
    path = np.asarray(path)
    for p, (i, j) in enumerate(path[:-2]):
        v1 = path[p + 1] - path[p]
        v2 = path[p + 1] - path[p + 2]
        v1_norm = np.linalg.norm(v1)
        v2_norm = np.linalg.norm(v2)
        angle = np.arccos(np.dot(v1, v2))
        if angle < np.pi:
            pass


# Questions:
## altitude leads to more costs for pylons? because they are higher?

# height profile constraints:
## simply exclude edges which cannot be placed --> only works when iterating over edges

## Angle constraints:
# * line graph
# * path straighening toolbox


In [ ]:
arr = np.arange(0,81,1).reshape((9,9))
arr

In [ ]:
shifts = get_half_donut(3, 8)
kernel, posneg = get_kernel(shifts)

In [ ]:
kernel[14]

In [ ]:
convolve(arr, kernel[14], 1)

In [ ]:
from scipy.signal import convolve2d

In [ ]:
def convolve_faster(img, f, neg):
    k_size = len(f)
    # a = np.pad(img, ((0, k_size-1), (0, k_size-1)))
    if neg:
        padded = np.pad(img, ((0, k_size - 1), (k_size - 1, 0)))
    else:
        padded = np.pad(img, ((0, k_size - 1), (0, k_size - 1)))
    
    s = f.shape + tuple(np.subtract(padded.shape, f.shape) + 1)
    strd = np.lib.stride_tricks.as_strided
    subM = strd(padded, shape = s, strides = padded.strides * 2)
    return np.einsum('ij,ijkl->kl', f, subM)

In [ ]:
arr2 = convolve_faster(arr, kernel[14], 1) # [5:-2, 2:-5] # 8x8
# [3:-6, 9:] # for 10x10 kernel, 1:-3,3:-1] # for 5x5 kernel

In [ ]:
arr2

In [ ]:
def convolve_faster(img, kernel, neg):
    k_size = len(kernel)
    print(k_size)
    # print(k_size)
    # if neg:
    #     padded = np.pad(img, ((0, k_size - 1), (k_size - 1, 0)))
    # else:
    #     padded = np.pad(img, ((0, k_size), (0, k_size)))
    padded = np.pad(img, ((0, k_size), (0, k_size)))
    print(padded.astype(int))
    print(kernel)
    return convolve2d(padded, kernel, mode ="same")[:-k_size-4, :-k_size-4]
# 8x8: [1:-k_size+1, 1:-k_size+1] 
# 7x7 [2:-k_size+2, :-k_size]
# 6x6 [3:-k_size+3, 1:-k_size+1]
# 5x5 

In [ ]:
arr2

In [ ]:
kernel[33].shape